In [29]:
import polars as pl

kp = pl.read_parquet('kjoretoy_preppet')

df_ant_el_2022 = kp.filter(
    (pl.col('tekn_reg_f_g_n').cast(str).str.contains('2022')) 
    & (pl.col('Er_Elbil') == True)
)

print(f"Antall Elbiler registrert i 2022 er: {df_ant_el_2022.height}")

Antall Elbiler registrert i 2022 er: 37249


In [53]:
antall_Elbil_2022 = (df_ant_el_2022.height)

df_prosent = kp.select(
    PersBil_Prosent_Er_El = (pl.col('tekn_reg_f_g_n').len() / antall_Elbil_2022).sum().round(2)
)

totaleBiler = kp.select(pl.col('tekn_reg_f_g_n').len())
HER ER DET NOE FEEEEEEIL
print(f"Antall biler totalt: {totaleBiler}  Antall elbiler: {antall_Elbil_2022}  Prosent: {prosent}")
df_prosent

AttributeError: 'int' object has no attribute 'height'